In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '45447f0361d8',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '4IrRB9I_QtSc9ZaomU5o5g',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [2]:
path = '/opt/ml/data/wikipedia_documents.json'

In [3]:
import json #json 수정하려고 열음
with open(path, 'r') as f:
    wiki = json.load(f)

In [4]:
for i in range(len(wiki)):
    del wiki[str(i)]['corpus_source']# 필요없는거 삭제
    del wiki[str(i)]['domain']
    del wiki[str(i)]['author']
    del wiki[str(i)]['html']
    del wiki[str(i)]['document_id']
    del wiki[str(i)]['url']

In [5]:
wiki['0']['text']

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [7]:
import re
for i in range(len(wiki)):
    temp = wiki[f'{i}']['text']
    temp = re.sub('[#*~"\"]','',temp)
    temp = temp.split('\n')
    temp = [s.strip() for s in temp]
    temp = list(filter(None, temp))
    temp = ' '.join(temp)
    title_token = ''.join(['[',wiki[f'{i}']['title'], ']'])
    temp = ' '.join([title_token, temp])
    wiki[f'{i}']['text'] = temp

In [8]:
wiki['0']

{'text': '[나라 목록] 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 국가 문서에서 설명하고 있다.',
 'title': '나라 목록'}

In [9]:
titles = []
for doc in wiki:
    titles.append(wiki[doc]['title'])
titles = list(set(titles))

title_list = []
for i, word in enumerate(titles):

    word_list = []
    # word_list.append(re.sub('\s', '', word))
    tmp = re.sub('[()]', '', word).split()
    for w in tmp:
        w = re.sub('~', '-', w)
        w = re.sub('/', '|', w)
        if len(w) == 1:
            continue
        elif re.search('[가-힣ㄱ-ㅎㅏ-ㅣ]', w) == None:
            continue
        elif re.search('년', w):
            continue
        else:
            title_list.append(w)

title_list = list(set(title_list))
if '' in title_list:
    title_list.pop(title_list.index(''))

In [10]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type": "stop",
                        "stopwords_path": "stopkr.txt" # stopkr.txt 파일이 /etc/elasticsearch 에 있어야 함
                    }
                },
                "tokenizer": {
                    "my_nori_tokenizer": {
                        "type": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "user_dictionary_rules": title_list
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "my_nori_tokenizer", # 위에서 정의한 my_nori_tokenizer
                        "decompound_mode": "mixed",
                        "filter" : ["my_stop_filter"] # 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            # "dynamic": "strict",
            "properties": {
                # "title":{"type": "text", "analyzer": "nori_analyzer"},
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "document_id": {"type": "long"}
            }
        }
    }

In [11]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [12]:
import time
import tqdm

for doc_id, doc in tqdm.tqdm(wiki.items(),total=len(wiki)):
    
    es.index(index=INDEX_NAME, id=doc_id, body=doc)
    # time.sleep(0.1) # 문서집어넣기 time슬립너어야 에러 안걸림 (너무빨리 문서를 호출해서)

100%|██████████| 60613/60613 [04:14<00:00, 237.86it/s]


In [13]:
import pprint
# doc = es.get(index=INDEX_NAME, id=60000)#
# pprint.pprint(doc)

In [14]:
tv = es.termvectors(index=INDEX_NAME, id=2, body={"fields" : ["content","title"]})

In [15]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 6683,
                                                 'sum_doc_freq': 10908,
                                                 'sum_ttf': 10913},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 13}


In [16]:
from datasets import load_dataset

In [17]:
import datasets
import pandas as pd
import re
import pandas as pd
from datasets import load_from_disk
trainpath = "/opt/ml/data/train_dataset"
testpath = '/opt/ml/data/test_dataset'
datasets = load_from_disk(testpath)
datasets

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [18]:
query = datasets['validation']['question']
# ground_truth = datasets['validation']['context']
id = datasets['validation']['id']
len(query)

600

In [19]:
id[0]

'mrc-1-000653'

In [20]:
correct = 0
total = []
for i, q in enumerate(query):
    q = q.replace('~','-')
    res = es.search(index=INDEX_NAME, q=q, size=15)
    # print('query : ', q)
    # print('id : ', id[i])
    # print(res)
    tmp = {
        'id':id[i],
        'text1':res['hits']['hits'][0]['_source']['text'],
        'text2':res['hits']['hits'][1]['_source']['text'],
        'text3':res['hits']['hits'][2]['_source']['text'],
        'text4':res['hits']['hits'][3]['_source']['text'],
        'text5':res['hits']['hits'][4]['_source']['text'],
        'text6':res['hits']['hits'][5]['_source']['text'],
        'text7':res['hits']['hits'][6]['_source']['text'],
        'text8':res['hits']['hits'][7]['_source']['text'],
        'text9':res['hits']['hits'][8]['_source']['text'],
        'text10':res['hits']['hits'][9]['_source']['text'],
        'text11':res['hits']['hits'][10]['_source']['text'],
        'text12':res['hits']['hits'][11]['_source']['text'],
        'text13':res['hits']['hits'][12]['_source']['text'],
        'text14':res['hits']['hits'][13]['_source']['text'],
        'text15':res['hits']['hits'][14]['_source']['text'],
    }
    total.append(tmp)
    
    # context = res['hits']['hits']
    # print(q)
    # print(context)
    # print('='*100)


# print('% : ', correct/len(query))

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [21]:
import pandas as pd
df = pd.DataFrame(total)
df


,id,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,text12,text13,text14,text15
0,mrc-1-000653,[더크 젠틀리의 성스러운 탐정 사무소] 더크 젠틀리의 성스러운 탐정사무소의 줄거리는...,[글리제 581 g] 글리제 581 g는 천칭자리 방향으로 지구로부터 약 20.5광...,[국제천문연맹의 행성의 정의] 이 정의는 엄밀해야 할 정의가 모호성을 띄고 있다는 ...,"[초저온 왜성] 초저온 왜성의 개념은 1997년 데이비 커크패트릭, 토드 헨리, 마...",[화가자리 베타 운동성단] 화가자리 베타 운동성단은 지구에서 가깝고 젊은 운동성단이...,"[HD 40307] HD 40307은 분광형 K의 주계열성으로, 남반구의 화가자리 ...",[HD 189733] HD 189733을 도는 행성은 HD 189733 b가 확인되...,[우주선] 우주선(宇宙船)은 우주를 다니도록 만들어진 탈것 또는 장치이다. 준궤도 ...,[우주선] 우주선(宇宙船)은 우주를 다니도록 만들어진 탈것 또는 장치이다. 준궤도 ...,[용암 행성] 용암 행성 대다수는 어머니 별을 아주 가까이에서 돌고 있는 암석 행성...,[안드로메다자리 웁실론] 안드로메다자리 웁실론(υ Andromedae / υ And...,[니콜라우스 코페르니쿠스] 코페르니쿠스가 주장한 태양중심설은 현재의 태양계의 구조와...,[니콜라우스 코페르니쿠스] 코페르니쿠스가 주장한 태양중심설은 현재의 태양계의 구조와...,[랄랑드 21185] 이 항성의 시선속도는 매우 일정해서 천문학자이자 행성 사냥꾼 ...,[글리제 581 e] e가 발견되기 전까지 글리제 581 계(系)에는 이미 3개의 ...
1,mrc-1-001113,[오퍼레이션7] 냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨...,[일본 선수권 시리즈] 2005년(제56회 / 지바 롯데 마린스 대 한신 타이거스 ...,[내셔널리그 2010] 내셔널리그 2010(대한생명의 후원을 받아 대한생명 내셔널리...,[카를루스 아우베르투 파헤이라] 2002년 FIFA 월드컵이 끝나고 브라질의 감독으...,[도펠죌트너] 도펠죌트너(Doppelsöldner두 배로 받는 용병)는 16세기 독...,[옌볜 푸더] 1955년 8월 1일에 한인 위주로 구성된 《길림성축구팀》(지린축구단...,[게하드 그리샤] 그리샤는 2008년에 FIFA 국제 심판이 됐다. 그리샤는 201...,[그리넬 칼리지] 1843년에 매사추세츠주의 앤도버 신학교에서 교육을 받은 회중 교...,"[유동화전문회사] 유동화회사는 자산유동화 업무를 새롭게 시작하거나, 금융기관에서 자...",[크리스 니코스키] SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고...,[미하우 칼레츠키] 1899년 폴란드의 우치(Łódź)에서 태어났다. 그는 폴리테크...,[카스카의 작중 행적] 모든 걸 망쳐 버리고 내게서 모든 걸 빼앗아간 녀석인데......,[코리 리오단] 2014년 1월 10일 KBO 리그 LG 트윈스와 계약하며 2013...,[박제언] 대한민국 평창이 2018년 동계 올림픽의 개최국이 되면서 대한민국은 스키...,[일본 선수권 시리즈] 일본 시리즈가 개최되는 10월11월에는 기후적으로 춥다는 것...
2,mrc-0-002191,"[엥겔베르트 돌푸스] 1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌...",[엥겔베르트 돌푸스] 돌푸스는 니더외스터라이히주의 텍싱탈에서 싱글맘이자 신실한 가톨...,[이용수 (1928년)] 1928년 경상북도 성주에서 출생하여 지난날 한때 경상북도...,[토머스 A. 헨드릭스] 1884년 민주당원들은 1856년 이래 자신들의 첫 대통령...,[요한 요아힘 베커] 1666년 베커는 마인츠 대학교의 의학 강사로 취직할 수 있었...,[조지 S. 바우트웰] 바우트웰은 1869년 율리시스 S. 그랜트 대통령으로부터 재...,[금전신탁] 금전신탁의 운용방법은 특정한 특정(特定) 금전신탁과 특정하지 않은 불특...,[죠죠의 기묘한 모험 All Star Battle] ; 디오의 성 : 1부의 무대....,[웨스트민스터궁] 1834년 10월 16일에 문서를 태우는 용도로 사용되던 화로에서...,"[요숭] 650년 섬주(陝州) 협석(硤石)에서 요의(姚懿, 요선의姚善意)의 아들로 ...",[SOPA와 PIPA 반대 시위] 지식재산권 보호 법안|온라인 저작권 침해 금지 법...,[2011년 캐나다 연방 선거] 2011년 캐나다 연방 선거 또는 제41대 캐나다 ...,[콘라트 3세 (부르군트)] 부르군트의 콘라트 3세(Conrad III/Konrad...,[수페르코파 이탈리아나 2018] 대회 후 여러 어두운 면들이 있고 전제군주제 국가...,[영국령 버진아일랜드의 코로나19 범유행] 오랜 기간 동안 확진자가 나오지 않다가 ...
3,mrc-0-003951,[서구화] 유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 ...,[아오테아로아] 아오테아로아(Aotearoa)는 마오리어에서 뉴질랜드를 가리키는 말...,[아오테아로아] 아오테아로아(Aotearoa)는 마오리어에서 뉴질랜드를 가리키는 말...,"[멕시코의 언어] 농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리포...",[멕시코의 언어] 멕시코의 언어는 다양하고 그 중 스페인어가 가장 널리 쓰인다. 멕...,[세계농아인연맹] 세계농아인연맹(World Federation of the Deaf...,"[다중심언어] 영어는 발음, 어휘, 철자법 등이 영국, 북미, 영어를 사용하는 아프...",[윌리엄 홉슨] 와이탕이 조약 1840년 영국 해군 장교 윌리엄 홉슨이 총독으로 뉴...,[로버트 피츠로이] 뉴질랜드 초대 총독 윌리엄 홉슨은 1842년에 사망했다. 뉴질랜...,[미국의 문화] 미국의 언어 미국은 연방 정부 차원에서 따로 공용어를 지정하지 않고...,"[바누아투] 국민의 대다수가 멜라네시아계에 속한다. 솔로몬 제도, 오스트레일리아, ...","[바누아투] 국민의 대다수가 멜라네시아계에 속한다. 솔로몬 제도, 오스트레일리아, ...",[국제 SIL] 국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매기...,[국제 SIL] 국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매기...,[홍콩식 영어] 영어는 홍콩의 공용어 중 하나이다. 그런데 현지 홍콩 사람들의 약 ...
4,mrc-1-001272,[보응우옌잡] 1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파...,[깟비 국제공항] 제1차 인도차이나 전쟁 중에는 프랑스 공군의 깟비 공군기지로 이용...,[러셀 법정] 러셀 법정은 1966년 영국의 철학자 버트런드 러셀이 베트남 전쟁에서...,[즈엉반민] 그는 1940년대에 자신의 군사 경력을 시작하여 프랑스 식민지 육군에 ...,[나산 공항] 공항이 폐쇄됨에 따라 현재 나산 공항에 대한 예약된 운항 서비스는 없...,"[타이빈성] 과거 홍강 삼각주의 중심부에 자리 잡고 있었지만, 과거에는 타이빈성은 ...",[베트남의 역사] 제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프랑...,[베트남의 역사] 제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프랑...,[명호작전] 이 작전에 관해서는 연합군의 침공 외의 전투는 일어나지 않았다. 일본군...,[세키가하라 전투 (소설)] 《세키가하라》(関ヶ原)는 시바 료타로의 역사소설이다. ...,"[까오반비엔] 까오반비엔(v=Cao Văn Viên|hn=高文園, 1921년 12월...",[웰링턴의 승전] 웰링턴의 승전 또는 비토리아 전투(Wellingtons Sieg ...,[제1차 세계 대전] 세르비아 전역 (제1차 세계 대전) 오스트리아가 침공하면서 8...,[제1차 세계 대전] 세르비아 전역 (제1차 세계 대전) 오스트리아가 침공하면서 8...,[아우구스트 빌헬름 폰 브라운슈바이크볼펜뷔텔베페른 공작] 아우구스트 빌헬름은 171...
...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [22]:
df.to_csv('../add_context_test_dataset.csv')